## Version 1

In [1]:
def get_pins(name, pinA = 'haveA', pinB = 'haveB'):
    if pinA == 'needA':
        pinA = input("Enter Pin A input for gate "+name+"-->")
    if pinB == 'needB':
        pinB = input("Enter Pin B input for gate "+name+"-->")
    return [pinA, pinB]

def and_gate(name, pinA = 'needA', pinB = 'needB'):
    inputs = get_pins(name,pinA,pinB)
    return inputs[0] and inputs[1]

def or_gate(name, pinA = 'needA', pinB = 'needB'):
    inputs = get_pins(name,pinA,pinB)
    return inputs[0] or inputs[1]

def not_gate(name, pinA = 'needA'):
    inputs = get_pins(name,pinA)
    return (1 - int(inputs[0]))

In [2]:
def main():
    return not_gate('g4', or_gate('g3',and_gate('g2'),and_gate('g1')))

In [3]:
main()

Enter Pin A input for gate g2-->1
Enter Pin B input for gate g2-->0
Enter Pin A input for gate g1-->1
Enter Pin B input for gate g1-->0


1

## Version 2

In [4]:
class LogicGate:

    def __init__(self,n):
        self.name = n
        self.output = None

    def getLabel(self):
        return self.name

    def getOutput(self):
        self.output = self.performGateLogic()
        return self.output


class BinaryGate(LogicGate):

    def __init__(self,n):
        super(BinaryGate, self).__init__(n)

        self.pinA = None
        self.pinB = None

    def getPinA(self):
        if self.pinA == None:
            return int(input("Enter Pin A input for gate "+self.getLabel()+"-->"))
        else:
            return self.pinA.getFrom().getOutput()

    def getPinB(self):
        if self.pinB == None:
            return int(input("Enter Pin B input for gate "+self.getLabel()+"-->"))
        else:
            return self.pinB.getFrom().getOutput()

    def setNextPin(self,source):
        if self.pinA == None:
            self.pinA = source
        else:
            if self.pinB == None:
                self.pinB = source
            else:
                print("Cannot Connect: NO EMPTY PINS on this gate")


class AndGate(BinaryGate):

    def __init__(self,n):
        BinaryGate.__init__(self,n)

    def performGateLogic(self):

        a = self.getPinA()
        b = self.getPinB()
        if a==1 and b==1:
            return 1
        else:
            return 0

class OrGate(BinaryGate):

    def __init__(self,n):
        BinaryGate.__init__(self,n)

    def performGateLogic(self):

        a = self.getPinA()
        b = self.getPinB()
        if a ==1 or b==1:
            return 1
        else:
            return 0

class UnaryGate(LogicGate):

    def __init__(self,n):
        LogicGate.__init__(self,n)

        self.pin = None

    def getPin(self):
        if self.pin == None:
            return int(input("Enter Pin input for gate "+self.getLabel()+"-->"))
        else:
            return self.pin.getFrom().getOutput()

    def setNextPin(self,source):
        if self.pin == None:
            self.pin = source
        else:
            print("Cannot Connect: NO EMPTY PINS on this gate")


class NotGate(UnaryGate):

    def __init__(self,n):
        UnaryGate.__init__(self,n)

    def performGateLogic(self):
        if self.getPin():
            return 0
        else:
            return 1


class Connector:

    def __init__(self, fgate, tgate):
        self.fromgate = fgate
        self.togate = tgate

        tgate.setNextPin(self)

    def getFrom(self):
        return self.fromgate

    def getTo(self):
        return self.togate

In [5]:
def main():
   g1 = AndGate("G1")
   g2 = AndGate("G2")
   g3 = OrGate("G3")
   g4 = NotGate("G4")
   c1 = Connector(g1,g3)
   c2 = Connector(g2,g3)
   c3 = Connector(g3,g4)
   print(g4.getOutput())

In [69]:
main()

Enter Pin A input for gate G1-->1
Enter Pin B input for gate G1-->1
Enter Pin A input for gate G2-->0
Enter Pin B input for gate G2-->1
0


In [30]:
class Logic:
    def __and__(self, other):
        return lambda name: AndGate(name, [self, other])
        
    def __or__(self, other):
        return lambda name: OrGate(name, [self, other])
    
    def __invert__(self):
        return lambda name: NotGate(name, [self])


class Pin(Logic):
    def val(self, i, for_gate):
        return int(input(f"Enter Pin {'ABCDE'[i-1]} for gate {for_gate}: "))


class Gate(Logic):
    def __init__(self, name, inputs=None):
        self.name = name
        if inputs is None:
            inputs = [Pin() for _ in range(self.arity)]
        self.inputs = inputs
     
    def input_vals(self):
        return [x.val(i, self.name) for i, x in enumerate(self.inputs, 1)]
        
    def val(self, *args):
        return self(*self.input_vals())

        
class AndGate(Gate):
    arity = 2
                         
    def __call__(self, x, y):
        return int(x == 1 and y == 1)
    
    
class OrGate(Gate):
    arity = 2
                         
    def __call__(self, x, y):
        return int(x == 1 or y == 1)
    
    
class NotGate(Gate):
    arity = 1
                         
    def __call__(self, x):
        return 1 - x

In [15]:
g1 = AndGate("G1", [Pin(), Pin()])
g2 = AndGate("G2", [Pin(), Pin()])
g3 = OrGate("G3", [g1, g2])
g4 = NotGate("G4", [g3])

In [16]:
g1 = (Pin() & Pin())("g1")
g2 = (Pin() & Pin())("g2")
g3 = (g1 | g2)("g3")
g4 = (~g3)("g3")

In [17]:
g4 = (~((Pin() & Pin())("g1") | (Pin() & Pin())("g2"))("g3"))("g4")

In [32]:
g1 = AndGate("G1")
g2 = AndGate("G2")
g3 = OrGate("G3", [g1, g2])
g4 = NotGate("G4", [g3])

In [33]:
g4.val()

Enter Pin A for gate G1: 0
Enter Pin B for gate G1: 1
Enter Pin A for gate G2: 0
Enter Pin B for gate G2: 1


1

In [34]:
g4 = (~(AndGate("g1") | AndGate("g2"))("g3"))("g4")

In [19]:
g3(0, 1)

1

In [20]:
g3(0, 0)

0